In [31]:
import csv
import pandas as pd

### Data Cleaning

##### Read the data into Pandas Dataframes.

In [32]:
page_data_file_path = "./page_data.csv"
wpds_data_file_path = "./WPDS_2018_data.csv"
page_data_df = pd.read_csv(page_data_file_path)
wpds_df = pd.read_csv(wpds_data_file_path)

In [33]:
page_data_df.head()

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070
4,Template:Namibia-politician-stub,Namibia,391862409


In [34]:
wpds_df.head()

,Geography,Population mid-2018 (millions)
0,AFRICA,"1,284"
1,Algeria,42.7
2,Egypt,97
3,Libya,6.5
4,Morocco,35.2


##### Clean page_data by removing the pages which represent templates.

In [35]:
is_template = page_data_df['page'].str.match('Template:')
page_data_cleaned_df = page_data_df[~is_template]

##### Clean wpds data by removing the rows representing cumulative regions or continents.

In [36]:
wpds_df["is_continent"] = wpds_df.Geography.str.isupper()
wpds_countries_df = wpds_df[~wpds_df["is_continent"]]
wpds_continents_df = wpds_df[wpds_df["is_continent"]]

##### Showing the wpds rows corresponding to Cumulative regions (continents).

In [37]:
wpds_continents_df

,Geography,Population mid-2018 (millions),is_continent
0,AFRICA,"1,284",True
56,NORTHERN AMERICA,365,True
59,LATIN AMERICA AND THE CARIBBEAN,649,True
95,ASIA,"4,536",True
144,EUROPE,746,True
189,OCEANIA,41,True


### Getting article quality predictions.

In [ ]:
TODO